In [3]:
import AmazonReviews

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
%load_ext autoreload
%autoreload 2

In [36]:
ar = AmazonReviews.AmazonReviews()

PATH = '../data/amazon_reviews_us_Toys_v1_00.tsv'
ar.load_data(PATH)

ar.calc_trend_score(review_days=30)

ar.create_observations()

ar.create_train_test_split(train_reduction=.01)

Read from pickle...


Review the distribution of the first review data. 1/1/2014 is the most popular. May need to move the cutoff date.

2014-01-01    0.030230
2014-01-02    0.029510
2014-01-03    0.026328
2014-01-04    0.016026
2014-01-07    0.014318

In [7]:
ar.reviews_selected_df.min_review_date.value_counts(normalize=True).sort_values(ascending = False).head()

2014-01-01    0.030230
2014-01-02    0.029510
2014-01-03    0.026328
2014-01-04    0.016026
2014-01-07    0.014318
Name: min_review_date, dtype: float64

In [8]:
ar.product_trend_df[ar.product_trend_df.trend == 1].describe()

,count,median,std,orig_std,review_success,trend_score,trend
count,5469.000000,5469.000000,5469.000000,5469.000000,5469.000000,5469.000000,5469.0
mean,73.554032,10.885160,2.182931,2.114140,11.285563,0.999998,1.0
std,81.650357,0.954892,1.027999,1.149140,9.822584,0.000003,0.0
min,8.000000,5.196152,0.463900,0.000000,5.964506,0.999987,1.0
25%,30.000000,11.180340,1.386188,1.386188,6.869424,0.999998,1.0
50%,55.000000,11.180340,2.382850,2.382850,8.231919,1.000000,1.0
75%,89.000000,11.180340,2.968471,2.968471,11.643701,1.000000,1.0
max,1268.000000,11.180340,4.365866,4.365866,167.967345,1.000000,1.0


## DTM Creation

Create DTM and only restrict features to English words in the `nltk.corpus.words`

In [37]:
from nltk.corpus import words, stopwords
from nltk import SnowballStemmer
import re

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.over_sampling import SMOTE
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, accuracy_score

In [38]:
words_corpus = set(words.words())
stop_words = set(stopwords.words())
# analyzer = CountVectorizer().build_analyzer()
stemmer = SnowballStemmer('english')

# should move this a tokenizer

def english_corpus(doc, tkpat=re.compile('\\b[a-z][a-z]+\\b')):
    return [stemmer.stem(w) for w in tkpat.findall(doc)]


# ar.add_dtm(CountVectorizer(stop_words='english', tokenizer=english_corpus, min_df=2),'count_1_gram')
# # ar.add_dtm(CountVectorizer(stop_words='english', min_df=0.005),'count_1_gram')

In [39]:
first_pipe = imbPipeline([('cnt_v', CountVectorizer(stop_words='english', tokenizer=english_corpus, min_df=2)),
                          ('lda', LatentDirichletAllocation(n_jobs=-1, learning_method='online', random_state=42)),
                          ('sm', SMOTE(random_state=42)),
                          ('ss', StandardScaler()),
                          ('log_reg', LogisticRegression(random_state=42))])

In [43]:
params = {
    'lda__n_components': Integer(5, 20),
    'lda__learning_decay': Real(0.5, 1),
    'log_reg__C': Categorical([0.001,0.01,0.1,1,10,100])
}

grid = BayesSearchCV(first_pipe, params, n_jobs=4, n_iter=5, scoring='precision', refit=True)

In [ ]:
ar.run_model(grid, 'mvp')

/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklear

In [235]:
features = ar.models['count_1_gram']['model'].get_feature_names()

len(features)

2169

In [236]:
features.sort(reverse=True)
features

['zipper',
 'zip',
 'zero',
 'yr',
 'younger',
 'young',
 'yoga',
 'yesterday',
 'yes',
 'yellow',
 'year',
 'yeah',
 'yay',
 'yard',
 'xxxl',
 'xxx',
 'xxs',
 'xxl',
 'xs',
 'xmas',
 'xlarg',
 'xl',
 'wrote',
 'wrong',
 'write',
 'wrist',
 'wrinkl',
 'wrap',
 'wrangler',
 'wow',
 'wouldnt',
 'wouldn',
 'worth',
 'worst',
 'wors',
 'worri',
 'worn',
 'world',
 'workout',
 'workmanship',
 'worker',
 'work',
 'wore',
 'word',
 'wool',
 'wont',
 'wonder',
 'won',
 'women',
 'woman',
 'witch',
 'wit',
 'wish',
 'wise',
 'wire',
 'wipe',
 'winter',
 'wing',
 'wine',
 'window',
 'wind',
 'win',
 'wiggl',
 'wig',
 'wife',
 'width',
 'wider',
 'wide',
 'wick',
 'white',
 'whi',
 'wherev',
 'whenev',
 'whatev',
 'wetsuit',
 'wet',
 'weren',
 'went',
 'weld',
 'welcom',
 'weird',
 'weight',
 'weigh',
 'weekend',
 'week',
 'wed',
 'websit',
 'web',
 'weav',
 'weather',
 'wearabl',
 'wear',
 'weak',
 'wayyyyy',
 'wayyyi',
 'wayyy',
 'way',
 'waterproof',
 'water',
 'watch',
 'wast',
 'wasnt',
 'wa

In [231]:
ar.models['count_1_gram']['X_train'].shape

(41003, 6468)

## Model Running Framework

1. Supervised model or pipeline is created
2. BayesSearchCV is configured
3. BayesSearchCV is fitted and scored and everything is logged.
4. Data frame maintained reporting F1, precision, accuracy, and recall (rows), and models (columns)
5. Perferably store the confusion matrix to plot heatmaps

In [24]:
grid.fit(ar.models['orig']['X_train'], ar.models['orig']['y_train'])

/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/anaconda3/lib/python3.6/site-packages/sklear

BayesSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cnt_v', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
       ...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_iter=5, n_jobs=4, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring='precision',
       search_spaces={'lda__n_components': Integer(low=5, high=20), 'lda__learning_decay': Real(low=0.5, high=1, prior='uniform', transform='identity'), 'log_reg__C': Categorical(categories=(0.001, 0.01, 0.1, 1, 10, 100), prior=None)},
       verbose=0)

In [29]:
print(grid.best_estimator_.named_steps)

{'cnt_v': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function english_corpus at 0x1a4a922a60>,
        vocabulary=None), 'lda': LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7366759453400837,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=17, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0), 'sm': SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=42,

In [25]:
y_pred = grid.predict(ar.models['orig']['X_train'])

print('F1', f1_score(ar.models['orig']['y_train'], y_pred))
print('Precision',precision_score(ar.models['orig']['y_train'], y_pred))
print('Recall', recall_score(ar.models['orig']['y_train'], y_pred))
print(confusion_matrix(ar.models['orig']['y_train'], y_pred))

F1 0.04144385026737968
Precision 0.021291208791208792
Recall 0.775
[[2634 1425]
 [   9   31]]


In [32]:
results = {
    'F1': f1_score(ar.models['orig']['y_train'], y_pred),
    'Precision': precision_score(ar.models['orig']['y_train'], y_pred),
    'Recall': recall_score(ar.models['orig']['y_train'], y_pred),
    'Accuracy': accuracy_score(ar.models['orig']['y_train'],y_pred)
}

In [35]:
import pandas as pd
pd.DataFrame(data=results.values(), index=results.keys())

ValueError: DataFrame constructor not properly called!

|metric|score|
---|---|
|F1| 0.03751465416178194|
|Precision| 0.01920768307322929|
|Recall| 0.8|

||Pred No| Pred Yes|
|---|---|---|
|Act No| 2425| 1634|
|Act Yes|8|   32|


In [19]:
ar.models['orig']['X_train'].shape

(4099,)

In [253]:
from sklearn.cluster import KMeans

In [254]:
km = KMeans(n_clusters=10)
X_train_new = km.fit_transform(X_train_new)

In [259]:
X_train_new

array([[0.025     , 0.30328256, 0.02500569, ..., 0.02500768, 0.02500069,
        0.02500012],
       [0.54997421, 0.05000069, 0.05000473, ..., 0.05      , 0.05      ,
        0.05      ],
       [0.00909091, 0.0090925 , 0.0090936 , ..., 0.00909194, 0.00909181,
        0.00909094],
       ...,
       [0.01666777, 0.01667307, 0.84998582, ..., 0.01666673, 0.01666685,
        0.01666713],
       [0.00588235, 0.06961397, 0.51398135, ..., 0.00588235, 0.00588249,
        0.00588249],
       [0.00416708, 0.00416713, 0.28236026, ..., 0.56351615, 0.00416685,
        0.00416674]])

In [194]:
ar.models['count_1_gram']['X_train'].shape

(410034, 202)

In [179]:
'he' in set(stopwords.words())

True

In [190]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [193]:
pyLDAvis.sklearn.prepare(lda, ar.models['count_1_gram']['X_train'], ar.models['count_1_gram']['model'])

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.066915  0.122164       1        1  25.395331
1     -0.209358 -0.174570       2        1  23.342925
4     -0.229193  0.017083       3        1  20.500229
3      0.186132  0.268299       4        1  17.312211
0      0.319334 -0.232975       5        1  13.449303, topic_info=     Category          Freq         Term         Total  loglift  logprob
term                                                                    
96    Default  65501.000000         love  65501.000000  30.0000  30.0000
72    Default  86041.000000        great  86041.000000  29.0000  29.0000
148   Default  74530.000000         size  74530.000000  28.0000  28.0000
70    Default  58594.000000         good  58594.000000  27.0000  27.0000
143   Default  41633.000000        shirt  41633.000000  26.0000  26.0000
130   Default  43721.000000      quality  43721.000000  25.0000  25.0000
120   Default  36813.000000      perfect  36813.000000  24.0000  24.0000
152   Default  47901.000000        small  47901.000000  23.0000  23.0000
44    Default  34519.000000        dress  34519.000000  22.0000  22.0000
115   Default  30940.000000      ordered  30940.000000  21.0000  21.0000
124   Default  23830.000000        price  23830.000000  20.0000  20.0000
92    Default  23100.000000         look  23100.000000  19.0000  19.0000
36    Default  25554.000000         cute  25554.000000  18.0000  18.0000
109   Default  46729.000000         nice  46729.000000  17.0000  17.0000
84    Default  27260.000000        large  27260.000000  16.0000  16.0000
62    Default  91946.000000          fit  91946.000000  15.0000  15.0000
113   Default  39134.000000          one  39134.000000  14.0000  14.0000
127   Default  18819.000000      product  18819.000000  13.0000  13.0000
21    Default  16371.000000          bra  16371.000000  12.0000  12.0000
157   Default  17186.000000        still  17186.000000  11.0000  11.0000
90    Default  19535.000000         long  19535.000000  10.0000  10.0000
186   Default  18334.000000          way  18334.000000   9.0000   9.0000
119   Default  19459.000000        pants  19459.000000   8.0000   8.0000
29    Default  33598.000000  comfortable  33598.000000   7.0000   7.0000
190   Default  44937.000000         well  44937.000000   6.0000   6.0000
132   Default  31098.000000       really  31098.000000   5.0000   5.0000
155   Default  18463.000000         soft  18463.000000   4.0000   4.0000
100   Default  28250.000000     material  28250.000000   3.0000   3.0000
174   Default  19811.000000         time  19811.000000   2.0000   2.0000
73    Default  11885.000000        happy  11885.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
96     Topic5  65500.251924         love  65501.061771   2.0062  -1.7390
143    Topic5  41632.466754        shirt  41633.277219   2.0062  -2.1922
120    Topic5  36812.256784      perfect  36813.068319   2.0062  -2.3152
73     Topic5  11884.973774        happy  11885.787711   2.0062  -3.4458
128    Topic5  10189.486691     purchase  10190.302573   2.0062  -3.5997
38     Topic5   8804.342537          day   8805.158832   2.0062  -3.7458
17     Topic5   8683.064485        black   8683.882445   2.0061  -3.7597
51     Topic5   8516.592827      exactly   8517.407306   2.0061  -3.7790
184    Topic5   8207.674124         wash   8208.490209   2.0061  -3.8160
145    Topic5   7227.001598       shorts   7227.814365   2.0061  -3.9432
195    Topic5   6475.155358         wore   6475.971084   2.0061  -4.0531
191    Topic5   5932.974410        white   5933.790445   2.0061  -4.1405
49     Topic5   5496.659092        every   5497.474041   2.0061  -4.2169
31     Topic5   4919.163104         cool   4919.975515   2.0061  -4.3279
24     Topic5   5760.203563         came   9169.536142   1.5413  -4.1701
72     Topic5  51922.087205        great  86041.215326   1.5012  -1.9713
17

In [204]:
import pickle

In [205]:
pickle.dump(ar, open('../data/ar.pkl', 'wb'))

PicklingError: Can't pickle <class 'AmazonReviews.AmazonReviews'>: it's not the same object as AmazonReviews.AmazonReviews

In [211]:
ar.models['orig']['X_train'].shape

(41003,)

## Pipelines

1. (CountVectorizer, TF-IDF) -> (LDA, PCA, NMF, Word2Vec) -> K-Means -> (Logistic Regression, Random Forest, Gradient Boost)
2. Sampling due to imbalanced classes (SMOTE, SMOTE->Tomek, SMOTE-> ENN) 